### Run This Part First

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!git clone https://github.com/mukund-ks/DeepLabV3-Segmentation

Cloning into 'DeepLabV3-Segmentation'...
remote: Enumerating objects: 528, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 528 (delta 117), reused 96 (delta 67), pack-reused 378
Receiving objects: 100% (528/528), 5.75 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (323/323), done.


In [2]:
!pip install openslide-python
!sudo apt-get install openslide-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openslide-python: filename=openslide_python-1.3.1-cp310-cp310-linux_x86_64.whl size=33550 sha256=625bc6891719f3f14e93dd6bfdd898c876f1a9cd633408030a57ab6d7a36bd91
  Stored in directory: /root/.cache/pip/wheels/79/79/fa/29a0087493c69dff7fd0b70fab5d6771002a531010161d2d97
Successfully built openslide-python
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]

In [3]:
import openslide
import numpy as np
import os
from PIL import Image
import cv2
import cv2
import numpy as np
import os
from PIL import Image
'''
def create_masks(image_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply binary thresholding
        _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

        # Morphological operations to clean up the mask
        kernel = np.ones((5,5), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

        # Save the mask
        mask_filename = os.path.join(output_dir, os.path.splitext(img_name)[0] + '_mask.tif')
        cv2.imwrite(mask_filename, binary)

'''
def extract_patches(svs_path, output_dir_img,output_dir_mask, patch_size=1024, overlap=50):
    slide = openslide.OpenSlide(svs_path)
    width, height = slide.dimensions
    images = []
    os.makedirs(output_dir_img, exist_ok=True)
    os.makedirs(output_dir_mask, exist_ok=True)
    x = 0
    while x < width:
        y = 0
        while y < height:
            patch = slide.read_region((x, y), 0, (patch_size, patch_size)).convert("RGB")
            patch_np = np.array(patch)
            patch_filename = os.path.join(output_dir_img, f"patch_{x}_{y}.png")
            Image.fromarray(patch_np).save(patch_filename)
            img = cv2.imread(patch_filename, cv2.IMREAD_COLOR)

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

            kernel = np.ones((5,5), np.uint8)
            binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
            binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

            mask_filename = os.path.join(output_dir_mask, os.path.splitext(patch_filename)[0] + '_mask.tif')
            cv2.imwrite(mask_filename, binary)

            y += patch_size - overlap
            images.append(patch_filename)
            if len(images)==8000:
              break
        x += patch_size - overlap

def resize_patches(img_pth):
  for img in os.listdir(img_pth):
    img = cv2.imread(img_pth, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256, 256))
    cv2.imwrite(img_pth, img)


def get_dim(svs_path):
  slide = openslide.OpenSlide(svs_path)
  width, height = slide.dimensions
  print(height,width)


import shutil

def mov_msk(img_pth,out_mask):
  for i in os.listdir(img_pth):
    if i.endswith('.tif'):
      shutil.move(os.path.join(img_pth,i),out_mask)




In [4]:
svs_paths = ['/content/drive/MyDrive/Project_3/AG-7435 #1.svs', '/content/drive/MyDrive/Project_3/AG-7435 #14.svs']
out_img = '/content/dataset/data/Image'
out_mask = "/content/dataset/data/Mask"

### Run this part if you don't have mask

In [ ]:
extract_patches(svs_paths[0], out_img,out_mask)
resize_patches( out_img)


In [ ]:
!mkdir dataset
!cp -r /content/drive/MyDrive/data /content/dataset

In [ ]:
!mkdir /content/dataset/eval_data

In [ ]:
mov_msk(out_img,out_mask)

In [ ]:
images = []
for img in os.listdir(out_img):
  images.append(img)
print(len(images))
masks = []
for img in os.listdir(out_mask):
  masks.append(img)
print(len(masks))

2580
2580


In [ ]:
images[0],masks[0]

('patch_0_32142.png', 'patch_26298_11688_mask.tif')

In [ ]:
import re

def extract_coordinates(filename, is_mask=False):
    """
    Extract x and y coordinates from the filename.
    If is_mask is True, handle the mask filename pattern.
    """
    if is_mask:
        match = re.search(r'patch_(\d+)_(\d+)_mask\.tif', filename)
    else:
        match = re.search(r'patch_(\d+)_(\d+)\.png', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        return None

def sort_images_and_masks(images, masks):
    """
    Sort images and masks based on their coordinates.
    """
    # Create pairs of (coordinates, filename)
    image_pairs = [(extract_coordinates(img), img) for img in images]
    mask_pairs = [(extract_coordinates(mask, is_mask=True), mask) for mask in masks]

    # Sort pairs based on coordinates
    image_pairs.sort()
    mask_pairs.sort()

    # Extract sorted filenames
    sorted_images = [img for _, img in image_pairs]
    sorted_masks = [mask for _, mask in mask_pairs]

    return sorted_images, sorted_masks


In [ ]:
sorted_images, sorted_masks = sort_images_and_masks(images, masks)

print("Sorted Images:", sorted_images[:10])
print("Sorted Masks:", sorted_masks[:10])

Sorted Images: ['patch_0_0.png', 'patch_0_974.png', 'patch_0_1948.png', 'patch_0_2922.png', 'patch_0_3896.png', 'patch_0_4870.png', 'patch_0_5844.png', 'patch_0_6818.png', 'patch_0_7792.png', 'patch_0_8766.png']
Sorted Masks: ['patch_0_0_mask.tif', 'patch_0_974_mask.tif', 'patch_0_1948_mask.tif', 'patch_0_2922_mask.tif', 'patch_0_3896_mask.tif', 'patch_0_4870_mask.tif', 'patch_0_5844_mask.tif', 'patch_0_6818_mask.tif', 'patch_0_7792_mask.tif', 'patch_0_8766_mask.tif']


In [ ]:
eval_img_pth = '/content/dataset/eval_data/Image'
eval_mask_pth = '/content/dataset/eval_data/Mask'


In [ ]:
!mkdir /content/dataset/eval_data/Image
!mkdir /content/dataset/eval_data/Mask

In [ ]:

for i in range(len(images)):
  shutil.copy(os.path.join(out_img,images[i]),eval_img_pth)
  shutil.copy(os.path.join(out_mask,masks[i]),eval_mask_pth)
  if i==1000:
    break

In [ ]:
eval_img = []
eval_mask = []
for img in os.listdir(eval_img_pth):
  eval_img.append(img)
print(len(eval_img))
for img in os.listdir(eval_mask_pth):
  eval_mask.append(img)
print(len(eval_mask))

1001
1001


In [ ]:
!cp -r /content/seg_data /content/drive/MyDrive

### Main Code start Work from below

In [5]:
!cp -r /content/drive/MyDrive/seg_data /content

In [34]:
# Utils file data
import os
from glob import glob
from typing import Union

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

H = 256
W = 256


def createDir(path: str) -> None:
    """Helper function to create a directory.

    Args:
        path (str): Path to create.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return


def loadData(path: str) -> list[str]:
    """Helper function to load data in lists from a directory.

    Args:
        path (str): Path to Data Directory.

    Raises:
        OSError: In case the Data Directory provided does not exist.

    Returns:
        list[str]: List of Images and Masks.
    """
    x = sorted(glob(os.path.join(path, "Image", "*png")))
    y = sorted(glob(os.path.join(path, "Mask", "*tif")))
    print(x[:5],y[:5])
    if len(x) == 0 or len(y) == 0:
        raise OSError("No Input in provided Directory.")
    return x, y


def shuffling(x: list[str], y: list[str]) -> list[str]:
    """Helper function to shuffle training set.

    Args:
        x (list[str]): Images
        y (list[str]): Masks

    Returns:
        list[str]: Shuffled list of images and masks.
    """
    x, y = shuffle(x, y, random_state=42)
    return x, y


def splitData(path: str) -> list[str]:
    """Helper function to split data into training and validation sets.

    Args:
        path (str): Path to Data Directory.

    Returns:
        list[str]: Paths to images and masks.
    """
    if not os.path.exists(os.path.join(path, "Image")) or not os.path.exists(
        os.path.join(path, "Mask")
    ):
        raise OSError(f"Incomplete Data Directory: {path}")

    X = sorted(glob(os.path.join(path, "Image", "*.png")))
    Y = sorted(glob(os.path.join(path, "Mask", "*.tif")))

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    return x_train, y_train, x_test, y_test


def createDirs(paths: tuple[str]) -> None:
    """Helper function to create a set of directories.

    Args:
        paths (tuple[str]): Tuple containing paths.
    """
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)
    return


def getMaskLen(y_pred: np.ndarray) -> Union[float, int]:
    """Determines a bounding box around the predicted mask and returns diagonal, horizontal and vertical length of the box in pixels

    Args:
        y_pred (numpy.ndarray): Binary mask from model prediction

    Returns:
        float | int: diagonal, horizontal & vertical length
    """
    binary_mask = np.array(y_pred, dtype=np.uint8)

    # Finding Contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Getting the Largest Contour
    try:
        largest_contour = max(contours, key=cv2.contourArea)
    except Exception as _:
        return 0.0, 0.0, 0.0

    # Getting the co-ordinates for smalles possible Bounding Box
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Calculating line lengths
    diagonal_length = round(np.sqrt(w**2 + h**2), 2)
    horizontal_length = round(w, 2)
    vertical_length = round(h, 2)

    return diagonal_length, horizontal_length, vertical_length


def saveResults(img: np.ndarray, mask: np.ndarray, y_pred: np.ndarray, save_img_path: str) -> None:
    """Helper function to save evaluation results as a single png, with provided image, ground truth, predicted mask and segmented output, from left to right.

    Args:
        img (np.ndarray): Input Image
        mask (np.ndarray): Ground Truth
        y_pred (np.ndarray): Predicted Mask
        save_img_path (str): Directory to save the results in.
    """
    line = np.ones((H, 10, 3)) * 128

    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    # mask = mask * 255

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

    masked_img = img * y_pred
    y_pred = y_pred * 255

    imgs = np.concatenate([img, line, mask, line, y_pred, line, masked_img], axis=1)
    cv2.imwrite(save_img_path, imgs)

    return


def tmp_cleanup() -> None:
    """Utility function for ens_train.py to delete temporary images and masks."""
    for dir in os.listdir("./tmp/data"):
        if not os.path.isdir(os.path.join("./tmp/data", dir)):
            continue
        for filename in os.listdir(os.path.join("./tmp/data", dir)):
            if not (filename.startswith("tmp") and filename.endswith(".png")):
                continue
            file_path = os.path.join("./tmp/data", dir, filename)
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        print(f"Deleted temporary files in ./tmp/data/{dir}\n")
    return


In [7]:
# data_processing code

import os

import cv2
import numpy as np
from albumentations import (
    ChannelShuffle,
    HorizontalFlip,
    RandomBrightnessContrast,
    RandomSizedCrop,
    Rotate,
    VerticalFlip,
)
from tqdm import tqdm

def augment_data(images: list, masks: list, save_path: str, augment: bool) -> None:
    """Function to apply augmentions and/or resize images to specific dimensions.

    Args:
        images (list): List containing paths to images.
        masks (list): List containing paths to masks.
        save_path (str): Path to save the new set.
        augment (bool): Opt-in to apply augmentations to the dataset.
    """
    H = 256
    W = 256

    for x, y in tqdm(zip(images, masks), total=len(images)):
        name = os.path.split(x)[1].split(".")[0]

        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

        if augment:
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = ChannelShuffle(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2 = augmented["mask"]

            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            aug = RandomBrightnessContrast(p=1.0)
            augmented = aug(image=x, mask=y)
            x4 = augmented["image"]
            y4 = augmented["mask"]

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=x, mask=y)
            x5 = augmented["image"]
            y5 = augmented["mask"]

            aug = RandomSizedCrop(min_max_height=(256, 256), width=256, height=256, p=1.0)
            augmented = aug(image=x, mask=y)
            x6 = augmented["image"]
            y6 = augmented["mask"]

            X = [x, x1, x2, x3, x4, x5, x6]
            Y = [y, y1, y2, y3, y4, y5, y6]

        else:
            X = [x]
            Y = [y]

        index = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (H, W))
            m = cv2.resize(m, (H, W))

            tmp_image_name = f"{name}_{index}.png"
            tmp_mask_name = f"{name}_{index}.tif"

            image_path = os.path.join(save_path, "Image", tmp_image_name)
            mask_path = os.path.join(save_path, "Mask", tmp_mask_name)

            if not cv2.imwrite(rf"{image_path}", i):
                print("Could not save image.")
                break
            if not cv2.imwrite(rf"{mask_path}", m):
                print("Could not save mask")
                break

            index += 1
    return


def processData(data_dir: str, augmentation: bool, split_data: bool) -> None:
    """Function to prepare dataset for training.

    Args:
        data_dir (str): Path to Data Directory.
        augmentation (bool): Opt-in to apply augmentations to provided data.
        split_data (bool): Opt-in to split data into training & validation set.

    Raises:
        OSError: In case the provided Data Directory does not exist.
    """
    np.random.seed(42)

    if not os.path.exists(data_dir):
        raise OSError("Directory does not exist.", data_dir)

    if split_data:
        x_train, y_train, x_test, y_test = splitData(data_dir)
    else:
        train_path = os.path.join(data_dir, "Train")
        x_train, y_train = loadData(path=train_path)

        test_path = os.path.join(data_dir, "Test")
        x_test, y_test = loadData(path=test_path)

    print(f"Train\t: {len(x_train)} - {len(y_train)}")
    print(f"Test\t: {len(x_test)} - {len(y_test)}")

    createDirs(
        (
            "./new_data/Train/Image/",
            "./new_data/Train/Mask/",
            "./new_data/Test/Image/",
            "./new_data/Test/Mask/",
        )
    )

    print("Creating New Training Set: ")
    augment_data(x_train, y_train, "./new_data/Train/", augment=augmentation)
    print("Creating New Testing Set: ")
    augment_data(x_test, y_test, "./new_data/Test/", augment=False)

    return



In [8]:
# Metrices
import numpy as np
import tensorflow as tf
from keras.layers import Flatten

SMOOTH = 1e-15


def iou(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + SMOOTH) / (union + SMOOTH)
        x = x.astype(np.float32)
        return x

    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


def dice_coef(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2.0 * intersection + SMOOTH) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + SMOOTH)


def calc_loss(model: tf.keras.Model) -> float:
    def dice_loss(y_true: np.ndarray, y_pred: np.ndarray, l2_weight: float = 1e-8):
        dice = dice_coef(y_true, y_pred)
        l2_loss = tf.add_n([l2_weight * tf.nn.l2_loss(var) for var in model.trainable_weights])
        return 1.0 - dice + l2_loss

    return dice_loss


# Evaluation Metrics


def eval_iou(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate Intersection over Union (IoU) for binary masks.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary mask (0 to 255).
    y_pred (numpy.ndarray): Predicted binary mask (0 or 1).

    Returns:
    float: IoU score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection

    x = (intersection + SMOOTH) / (union + SMOOTH)
    x = x.astype(np.float32)
    return x


def eval_dice_coef(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate the Dice Coefficient for binary masks.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary mask (0 to 255).
    y_pred (numpy.ndarray): Predicted binary mask (0 or 1).

    Returns:
    float: Dice Coefficient score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)

    if union == 0:
        return 1.0  # Handle the case where both masks are empty

    dice = (2.0 * intersection + SMOOTH) / (union + SMOOTH)
    return dice


def accuracy_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate the Accuracy score for binary classification.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary labels (0 to 255).
    y_pred (numpy.ndarray): Predicted binary labels (0 or 1).

    Returns:
    float: Accuracy score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    correct = np.sum(y_true == y_pred)
    total = y_true.size
    accuracy = correct / total

    return accuracy


def precision_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate the Precision score for binary classification.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary labels (0 to 255).
    y_pred (numpy.ndarray): Predicted binary labels (0 or 1).

    Returns:
    float: Precision score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))

    if TP + FP == 0:
        return 1.0  # case where both TP and FP are zero

    precision = TP / (TP + FP)

    return precision


def recall_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate the Recall (Sensitivity) score for binary classification.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary labels (0 to 255).
    y_pred (numpy.ndarray): Predicted binary labels (0 or 1).

    Returns:
    float: Recall (Sensitivity) score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    TP = np.sum((y_true == 1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    if TP + FN == 0:
        return 1.0  # case where TP and FN are zero

    recall = TP / (TP + FN)
    return recall


def f1_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Calculate the F1 Score for binary classification.

    Parameters:
    y_true (numpy.ndarray): Ground truth binary labels (0 to 255).
    y_pred (numpy.ndarray): Predicted binary labels (0 or 1).

    Returns:
    float: F1 Score.
    """
    y_true[y_true > 0] = 1  # normalize binary mask
    y_true = y_true.astype(np.float32)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    if precision + recall == 0:
        return 0.0  # Handle the case where both precision and recall are zero

    f1 = 2 * (precision * recall) / (precision + recall)
    return f1


In [9]:
"""
Model
Encoder : ResNet50, ResNet101, Xception or EfficientNetB5
Decoder : Deeplv3+
"""

import os

from keras.applications import EfficientNetB5, ResNet50, ResNet101, Xception
from keras.layers import (
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    # Dropout,
    GlobalAveragePooling2D,
    Input,
    Reshape,
    UpSampling2D,
)
from keras.models import Model

# from keras.regularizers import l2
from tensorflow.python.keras.engine.keras_tensor import KerasTensor

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"


def squeeze_and_excite(inputs: KerasTensor, ratio: int = 8) -> KerasTensor:
    """Function to apply Squeeze & Excitation to a feature map.

    Args:
        inputs (KerasTensor): Feature Map
        ratio (int, optional): Ratio for excitation in first dense layer. Defaults to 8.

    Returns:
        KerasTensor: Re-calibrated feature map.
    """
    init = inputs
    filters = init.shape[-1]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(
        filters // ratio,
        activation="relu",
        kernel_initializer="he_normal",
        use_bias=False,
    )(se)
    se = Dense(
        filters,
        activation="sigmoid",
        kernel_initializer="he_normal",
        use_bias=False,
    )(se)

    x = init * se
    return x


def ASPP(inputs: KerasTensor) -> KerasTensor:
    """Function to apply Atrous Spatial Pyramid Pooling on features from backbone.

    Args:
        inputs (KerasTensor): Features from backbone.

    Returns:
        KerasTensor: Features with better spatial context.
    """
    shape = inputs.shape
    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(inputs)
    y1 = Conv2D(256, 1, padding="same", use_bias=False, kernel_initializer="he_normal")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)
    y1 = squeeze_and_excite(y1)

    # 1x1 Convolution
    y2 = Conv2D(256, 1, padding="same", use_bias=False, kernel_initializer="he_normal")(inputs)
    y2 = BatchNormalization()(y2)
    # y2 = Dropout(0.5)(y2)
    y2 = Activation("relu")(y2)
    y2 = squeeze_and_excite(y2)

    # 3x3 Convolution, Dilation Rate - 6
    y3 = Conv2D(
        256, 3, padding="same", dilation_rate=6, use_bias=False, kernel_initializer="he_normal"
    )(inputs)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)
    y3 = squeeze_and_excite(y3)

    # 3x3 Convolution, Dilation Rate - 12
    y4 = Conv2D(
        256, 3, padding="same", dilation_rate=12, use_bias=False, kernel_initializer="he_normal"
    )(inputs)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)
    y4 = squeeze_and_excite(y4)

    # 3x3 Convolution, Dilation Rate - 18
    y5 = Conv2D(
        256, 3, padding="same", dilation_rate=18, use_bias=False, kernel_initializer="he_normal"
    )(inputs)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)
    y5 = squeeze_and_excite(y5)

    # 3x3 Convolution, Dilation Rate - 24
    y6 = Conv2D(
        256, 3, padding="same", dilation_rate=24, use_bias=False, kernel_initializer="he_normal"
    )(inputs)
    y6 = BatchNormalization()(y6)
    y6 = Activation("relu")(y6)
    y6 = squeeze_and_excite(y6)

    # 3x3 Convolution, Dilation Rate - 36
    y7 = Conv2D(
        256, 3, padding="same", dilation_rate=36, use_bias=False, kernel_initializer="he_normal"
    )(inputs)
    y7 = BatchNormalization()(y7)
    y7 = Activation("relu")(y7)
    y7 = squeeze_and_excite(y7)

    # 1x1 Convolution on the concatenated Feature Map
    y = Concatenate()([y1, y2, y3, y4, y5, y6, y7])
    y = Conv2D(256, 1, padding="same", use_bias=False, kernel_initializer="he_normal")(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y = squeeze_and_excite(y)

    return y


def createModel(modelType: str, shape: tuple[int] = (256, 256, 3)) -> Model:
    """Creates a Model with the specified backbone.

    Args:
        modelType (str): Choice of backbone. ResNet50, ResNet101, Xception or EfficientNetB5.
        shape (tuple[int]): Shape of input to the model. Defaults to (256, 256, 3).

    Returns:
        Model: Your DeepLabV3+ Model.
    """
    inputs = Input(shape)  # instantiating a tensor

    def get_xception():
        return Xception(weights="imagenet", include_top=False, input_tensor=inputs)

    def get_efficientb5():
        return EfficientNetB5(weights="imagenet", include_top=False, input_tensor=inputs)

    def get_resnet50():
        return ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)

    def get_resnet101():
        return ResNet101(weights="imagenet", include_top=False, input_tensor=inputs)

    model_mappings = {
        "Xception": {
            "base_model_func": get_xception,
            "block_name": "block13_sepconv2_act",
            "low_level_name": "block4_sepconv1_act",
        },
        "EfficientNetB5": {
            "base_model_func": get_efficientb5,
            "block_name": "block7c_activation",
            "low_level_name": "block2e_activation",
        },
        "ResNet101": {
            "base_model_func": get_resnet101,
            "block_name": "conv4_block23_out",
            "low_level_name": "conv2_block2_out",
        },
        "ResNet50": {
            "base_model_func": get_resnet50,
            "block_name": "conv4_block6_out",
            "low_level_name": "conv2_block2_out",
        },
    }

    model_params = model_mappings.get(modelType, {})
    base_model_func = model_params.get("base_model_func")
    base_model = base_model_func()
    block_name = model_params.get("block_name")
    low_level_name = model_params.get("low_level_name")

    image_features = base_model.get_layer(block_name).output

    if modelType == "Xception":
        image_features = Conv2D(
            1024, (1, 1), padding="same", kernel_initializer="he_normal", use_bias=False
        )(image_features)
        image_features = BatchNormalization()(image_features)
        image_features = Activation("relu")(image_features)

    if modelType == "EfficientNetB5":
        image_features = UpSampling2D((2, 2), interpolation="bilinear")(image_features)
        image_features = Conv2D(
            1024, (1, 1), padding="same", kernel_initializer="he_normal", use_bias=False
        )(image_features)
        image_features = BatchNormalization()(image_features)
        image_features = Activation("relu")(image_features)

    # High-Level Features
    x_a = ASPP(image_features)

    # Up-Sampling High-Level Features by 4
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)
    # x_a = Dropout(0.5)(x_a)

    # Low-Level Features
    x_b = base_model.get_layer(low_level_name).output
    if modelType == "Xception":
        x_b = UpSampling2D((2, 2), interpolation="bilinear")(x_b)
    if modelType == "EfficientNetB5":
        x_b = Conv2D(256, (1, 1), padding="same", kernel_initializer="he_normal", use_bias=False)(
            x_b
        )

    # 1x1 Convolution on Low-Level Features
    x_b = Conv2D(
        filters=48, kernel_size=1, padding="same", use_bias=False, kernel_initializer="he_normal"
    )(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation("relu")(x_b)
    x_b = squeeze_and_excite(x_b)

    # Concatenating High-Level and Low-Level Features
    x = Concatenate()([x_a, x_b])
    # x = Dropout(0.5)(x)

    # 3x3 Convolution on Concatenated Map
    x = Conv2D(
        filters=256, kernel_size=3, padding="same", use_bias=False, kernel_initializer="he_normal"
    )(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = squeeze_and_excite(x)

    # 3x3 Convolution on Concatenated Map
    x = Conv2D(
        filters=256, kernel_size=3, padding="same", use_bias=False, kernel_initializer="he_normal"
    )(x)
    x = BatchNormalization()(x)
    # x = Dropout(0.5)(x)
    x = Activation("relu")(x)
    x = squeeze_and_excite(x)

    # Final Up-Sampling by 4
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    x = Conv2D(1, 1)(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs, x)

    return model



In [10]:
# Trainer code
import os
from typing import Any

import cv2
import numpy as np
import tensorflow as tf
from keras.callbacks import (
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from keras.metrics import Accuracy, Precision, Recall
from keras.optimizers import Adam

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

H = 256
W = 256
LR = 2e-4

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]


def read_image(path: Any) -> Any:
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32)
    x = x / 255.0  # normalizing and standardizing image with Imagenet specifications
    x -= MEAN
    x /= STD
    return x


def read_mask(path: Any) -> Any:
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
    x = x / 255.0  # normalizing mask
    x = np.expand_dims(x, axis=-1)
    return x


def tf_parse(x: Any, y: Any) -> Any:
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


def tf_dataset(X: Any, Y: Any, batch: int = 2) -> Any:
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


def trainer(
    stop_early: bool,
    batches: int,
    epochs: int,
    modelType: str,
) -> None:
    """Function to train the model.

    Args:
        stop_early (bool): Opt-in to use Early-Stopping during training.
        batches (int): No. of batches.
        epochs (int): No. of epochs.
        modelType (str): Choice of backbone. ResNet50 or ResNet101.
    """
    np.random.seed(42)
    tf.random.set_seed(42)

    createDir("output")
    files_dir = "output"

    model_path = os.path.join(files_dir, "model.h5")
    csv_path = os.path.join(files_dir, "Epoch_Log.csv")

    train_path = os.path.join("./new_data", "Train")
    val_path = os.path.join("./new_data", "Test")

    x_train, y_train = loadData(train_path)
    x_train, y_train = shuffling(x_train, y_train)
    x_val, y_val = loadData(val_path)

    print(f"Train:\nImages: {len(x_train)}\tMasks: {len(y_train)}")
    print(f"Validation:\nImages: {len(x_val)}\tMasks: {len(y_val)}")

    train_dataset = tf_dataset(x_train, y_train, batch=batches)
    val_dataset = tf_dataset(x_val, y_val, batch=batches)

    model = createModel(shape=(H, W, 3), modelType=modelType)

    loss_fn = calc_loss(model=model)

    model.compile(
        loss=loss_fn,
        optimizer=Adam(LR),
        metrics=[dice_coef, iou, Recall(), Precision(), Accuracy()],
    )

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-8, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(log_dir="logs"),
    ]

    if stop_early:
        callbacks.append(
            EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False),
        )

    print(f"\nUsing {modelType} as Encoder{' with Early Stopping.' if stop_early else '.'}\n")

    model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=callbacks)
    return




In [15]:
# Evaluation code
import os
import traceback

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import load_model
from keras.utils import CustomObjectScope
from tqdm import tqdm

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

H = 256
W = 256

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

def model_iou(y_pred: np.ndarray, y_true: np.ndarray) -> float:
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + SMOOTH) / (union + SMOOTH)
        x = x.astype(np.float32)
        return x

    return tf.numpy_function(f, [y_true, y_pred], tf.float32)



def evaluator(eval_dir: str) -> None:
    """Function to evaluate the trained model.

    Args:
        eval_dir (str): Path to evaluation directory.

    Raises:
        OSError: In case the provided Evaluation Path does not exist.
    """
    np.random.seed(42)
    tf.random.set_seed(42)

    if not os.path.exists(eval_dir):
        raise OSError("Path does not exist.", eval_dir)

    createDir("eval_results")

    with CustomObjectScope({"iou": model_iou, "dice_coef": dice_coef, "dice_loss": calc_loss}):
        model = load_model("./output/model.h5")

    try:
        x_test, y_test = loadData(eval_dir)
    except Exception as _:
        traceback.print_exc()
        exit("No input available.")

    print(f"Test:\nImages: {len(x_test)}\tMasks: {len(y_test)}")

    SCORE = []
    for x, y in tqdm(zip(x_test, y_test), total=len(x_test)):
        name = os.path.split(x)[1].split(".")[0]

        image = cv2.imread(x, cv2.IMREAD_COLOR)
        image_resized = cv2.resize(image, dsize=(256, 256))
        x = image_resized / 255.0
        x -= MEAN
        x /= STD
        x = np.expand_dims(x, axis=0)

        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, dsize=(256, 256))

        y_pred = model.predict(x)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = (y_pred > 0.5).astype(np.int32)

        diagonal_len, horizontal_len, vertical_len = getMaskLen(y_pred)

        save_img_path = f"./eval_results/{name}.png"

        saveResults(image_resized, mask_resized, y_pred, save_img_path)

        mask_resized = mask_resized.flatten()
        y_pred = y_pred.flatten()

        acc_scr = accuracy_score(y_pred=y_pred, y_true=mask_resized)
        f1_scr = f1_score(y_pred=y_pred, y_true=mask_resized)
        recall_val = recall_score(y_pred=y_pred, y_true=mask_resized)
        precison_val = precision_score(y_pred=y_pred, y_true=mask_resized)
        iou = eval_iou(y_pred=y_pred, y_true=mask_resized)
        dice = eval_dice_coef(y_pred=y_pred, y_true=mask_resized)
        SCORE.append(
            [
                name,
                acc_scr,
                f1_scr,
                recall_val,
                precison_val,
                iou,
                dice,
                diagonal_len,
                horizontal_len,
                vertical_len,
            ]
        )

    score = [s[1:7] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1-Score: {score[1]:0.5f}")
    print(f"Recall: {score[2]:0.5f}")
    print(f"Precison: {score[3]:0.5f}")
    print(f"IoU: {score[4]:0.5f}")
    print(f"Dice Coefficient: {score[5]:0.5f}")

    df = pd.DataFrame(
        SCORE,
        columns=[
            "Image",
            "Accuracy",
            "F1",
            "Recall",
            "Precison",
            "IoU",
            "Dice Coeff",
            "Diagonal Length (px)",
            "Horizontal Length (px)",
            "Vertical Length (px)",
        ],
    )
    df.to_csv("./eval_results/Evaluation_Score.csv")

    return



In [42]:
def main(
    data_dir = '/content/seg_data/data',
    eval_dir =  '/content/seg_data/data',
    augmentation = False,
    split_data = True,
    stop_early =True,
    batch_size = 4,
    epochs = 1,
    model_type = "ResNet50",
) -> None:
    """
    A DeepLab V3+ Decoder based Binary Segmentation Model with choice of Encoders b/w ResNet101, ResNet50, Xception or EfficientNetB5.\n
    See: https://github.com/mukund-ks/DeepLabV3-Segmentation
    """
    try:
        print(f"\nData Pre-Processing Phase\n{'-'*10}")
        processData(data_dir=data_dir, augmentation=augmentation, split_data=split_data)
        print(f"\nTraining Phase\n{'-'*10}")
        trainer(
            stop_early=stop_early,
            batches=batch_size,
            epochs=epochs,
            modelType=model_type,
        )
        print(f"\nEvaluation Phase\n{'-'*10}")
        evaluator(eval_dir=eval_dir)
    except Exception as _:
        traceback.print_exc()

    return


In [17]:
if __name__ == "__main__":
    main()


Data Pre-Processing Phase
----------
Train	: 2064 - 2064
Test	: 516 - 516
Creating New Training Set: 


100%|██████████| 2064/2064 [01:50<00:00, 18.70it/s]


Creating New Testing Set: 


100%|██████████| 516/516 [00:29<00:00, 17.40it/s]



Training Phase
----------
./new_data/Train
./new_data/Test
Train:
Images: 2064	Masks: 2064
Validation:
Images: 516	Masks: 516

Using ResNet50 as Encoder with Early Stopping.

516/516 [==============================] - ETA: 0s - loss: 0.1663 - dice_coef: 0.8339 - iou: 0.7471 - recall_1: 0.8695 - precision_1: 0.8648 - accuracy: 0.0079
Epoch 1: val_loss improved from inf to 0.30937, saving model to output/model.h5


...

1/1 [==============================] - 0s 54ms/step


100%|█████████▉| 1000/1001 [03:25<00:00,  5.87it/s]

1/1 [==============================] - 0s 53ms/step


100%|██████████| 1001/1001 [03:25<00:00,  4.87it/s]


Accuracy: 0.58449
F1-Score: 0.61495
Recall: 0.98116
Precison: 0.59054
IoU: 0.57975
Dice Coefficient: 0.61495


In [41]:
x,y=loadData('/content/seg_data/eval_data')
print(x[1000],y[1000])

['/content/seg_data/eval_data/Image/patch_0_0.png', '/content/seg_data/eval_data/Image/patch_0_12662.png', '/content/seg_data/eval_data/Image/patch_0_13636.png', '/content/seg_data/eval_data/Image/patch_0_15584.png', '/content/seg_data/eval_data/Image/patch_0_16558.png'] ['/content/seg_data/eval_data/Mask/patch_0_0_mask.tif', '/content/seg_data/eval_data/Mask/patch_0_10714_mask.tif', '/content/seg_data/eval_data/Mask/patch_0_11688_mask.tif', '/content/seg_data/eval_data/Mask/patch_0_14610_mask.tif', '/content/seg_data/eval_data/Mask/patch_0_15584_mask.tif']
/content/seg_data/eval_data/Image/patch_974_35064.png /content/seg_data/eval_data/Mask/patch_974_974_mask.tif


In [31]:
# Manual testing
with CustomObjectScope({"iou": model_iou, "dice_coef": dice_coef, "dice_loss": calc_loss}):
    model = load_model("./output/model.h5")

image = cv2.imread('/content/patch_10714_15584_0.png')
print(image.shape)
image = cv2.resize(image, dsize=(256, 256))

image = np.expand_dims(image, axis=0)
preds = model.predict(image)

(256, 256, 3)
1/1 [==============================] - 2s 2s/step


In [32]:
preds[0].shape

(256, 256, 1)

In [33]:
cv2.imwrite('/content/mask.png', preds[0])

True